# Data and imports


In [4]:
import unicodedata
from pathlib import Path
import html
from sklearn.metrics.pairwise import cosine_similarity
import tqdm
import re
from tqdm.auto import tqdm  # Added tqdm import
from sentence_transformers import SentenceTransformer
import random
import numpy as np
import torch, platform
import polars as pl
import os
from pathlib import Path
from huggingface_hub import login
import os
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path
# PATH_TO_ITEMS = Path().cwd().parent / "data" / "modified_data" / "items.parquet"

ModuleNotFoundError: No module named 'google'

In [ ]:
# If using google colab

# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
path_to_data = Path.cwd() / "drive" / "MyDrive" / "ColabNotebooks" / "Диплом"
os.chdir(path_to_data)

In [6]:
# For MacOs to check if MPS is available

# If using mac
print("macOS:", platform.mac_ver()[0], "  PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("MPS built‑in:", torch.backends.mps.is_built())

# If using colab
print("Colab:", torch.__version__)
print("Cuda:", torch.cuda.is_available())


macOS:    PyTorch: 2.6.0+cu124
MPS available: False
MPS built‑in: False
Colab: 2.6.0+cu124
Cuda: True


In [7]:
# Specifing seed value for reproducibility

SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.set_random_seed(SEED)


In [8]:
items_df = pl.read_parquet("items.parquet")

# Checking if there is any null value in the DataFrame
null_sum = items_df.null_count().sum_horizontal()[0]

assert null_sum == 0, f"There are {null_sum} null values in the DataFrame"


# Preparation


## Keywords problem


- Проверим, насколько наши текущие ключевые слова подходят для кластеризации


In [ ]:
# Print samples of dataset's keywords
sample_dataset = (
    items_df.sample(2)
    .select("keywords", "countries", "title", "description")
    .rows(named=True)
)

for item in sample_dataset:
    print(f"Title: {item['title']}")
    print(f"Description: {item['description']}")
    print(f"Countries: {item['countries']}")
    print(f"Keywords: {item['keywords'][:150]}...")
    print("-" * 80)

Title: с прицепом
Description: Жизненная комедия про трудности и радости материнства. Сериал «С прицепом» основан на личной истории актрисы Фрэнки Шоу («Мистер Робот»), которая сыграла главную роль, а также приложила руку к сценарию и режиссуре проекта. В центре сюжета — двадцатилетняя девушка с годовалым ребенком на руках. У нее много проблем и забот, однако молодая мама уверена, что обязательно обретет счастье в личной жизни. Правда, парни ей попадаются в основном из робкого десятка: ребенок кажется им непреодолимой преградой в отношениях.
Countries: сша
Keywords: прицепом, 2017, США...
--------------------------------------------------------------------------------
Title: виктория
Description: Актриса Дженна Коулман, звезда вселенной Доктора Кто, в роли молодой британской королевы Виктории, занявшей престол в 1837 году и давшей имя целой эпохе.  Виктория приняла Британию в крайне плачевном состоянии. Ей было всего 18 лет, и рядом не было никого, кто был бы ей мудрым и верным наставн

- Во-первых, идет дубляция страны, что не хорошо. Зачем нам заниматься увеличением токенов, которые у нас повторяются в графе Стран
- Видно, что предложения по типу "борьба за выживание" разделены запятой, а не точкой с запятой

- Необходимо придумать более осмысленные ключевые слова!


In [ ]:
os.environ["HF_TOKEN"] = userdata.get("hug")
login(os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
DEVICE = "cuda"
MODEL = "google/gemma-3-4b-it"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map=DEVICE,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
model.to(DEVICE)

model = torch.compile(model, mode="max-autotune")

NameError: name 'model' is not defined

In [ ]:
SYSTEM_MSG = (
    "Ты — эксперт-аналитик фильмов и сериалов. "
    "Твоя задача — извлечь ключевых слова или короткие фразы, "
    "лучше всего описывающих данный контент.\n\n"
)


def build_user_prompt(row: dict) -> str:
    return (
        f"Название: {row.get('title', 'N/A')}\n"
        f"Оригинал: {row.get('title_orig', 'N/A')}\n"
        f"Жанры: {row.get('genres', 'N/A')}\n"
        f"Описание: {row.get('description', 'N/A')}\n\n"
        "Инструкции:\n"
        "1. Сконструируй до 8-10 ключевых слов фраз.\n"
        "2. Сосредоточься на темах, сеттинге, атмосфере.\n"
        "3. Избегай слишком общих слов (фильм, сериал, год, актёры).\n"
        "4. Вывод — ТОЛЬКО запятая-разделённый список.\n\n"
        "Ключевые слова:"
    )

In [ ]:
batch_size = 64
generated_keywords_list = []

all_items_dicts = items_df.to_dicts()

with torch.no_grad():
    for i in tqdm(range(0, len(all_items_dicts), batch_size), desc="Обработка батчей"):
        batch_dicts = all_items_dicts[i : i + batch_size]

        batch_prompts = []
        for row_dict in batch_dicts:
            user_prompt = build_user_prompt(row_dict)
            full_prompt = SYSTEM_MSG + user_prompt
            batch_prompts.append(full_prompt)

        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        ).to(DEVICE)

        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.3,
            # eos_token_id=tokenizer.eos_token_id
            # pad_token_id=tokenizer.pad_token_id
        )

        generated_texts_batch = tokenizer.batch_decode(
            outputs[:, inputs.input_ids.shape[1] :], skip_special_tokens=True
        )

        for idx, generated_text in enumerate(generated_texts_batch):
            original_item_dict = batch_dicts[idx]
            generated_keywords_list.append(
                {
                    "item_id": original_item_dict.get("item_id", None),
                    "original_description": original_item_dict.get("description"),
                    "generated_keywords": generated_text.strip(),
                }
            )


Обработка батчей:   0%|          | 0/250 [00:00<?, ?it/s]

- Случайно не то поле с id указал, поэтому перепроверяем генерацию


In [ ]:
if len(generated_keywords_list) != len(items_df):
    print(
        f"Длина списка ключевых слов ({len(generated_keywords_list)}) не совпадает с длиной DataFrame ({len(items_df)})."
    )
    print("Добавление столбца невозможно. Проверьте корректность данных.")
else:
    print(
        f"Длины совпадают: {len(items_df)}. Выполняется выборочная проверка содержимого..."
    )

    all_content_match = True

    indices_to_check = [0]
    if len(items_df) > 1:
        indices_to_check.append(len(items_df) - 1)
    if len(items_df) > 2:
        indices_to_check.append(len(items_df) // 2)
    if len(items_df) > 10:
        possible_random_indices = [
            i for i in range(1, len(items_df) - 1) if i not in indices_to_check
        ]
        if len(possible_random_indices) >= 3:
            indices_to_check.extend(random.sample(possible_random_indices, 3))
        elif possible_random_indices:
            indices_to_check.extend(
                random.sample(possible_random_indices, len(possible_random_indices))
            )

    indices_to_check = sorted(list(set(indices_to_check)))

    print(f"Проверяем индексы: {indices_to_check}")

    description_column_name_in_df = "description"

    if description_column_name_in_df not in items_df.columns:
        print(f"Столбец '{description_column_name_in_df}' не найден в items_df.")
        all_content_match = False
    else:
        for idx in indices_to_check:
            description_from_df = items_df.select(
                pl.col(description_column_name_in_df)
            )[idx, 0]
            description_from_list = generated_keywords_list[idx].get(
                "original_description"
            )
            match = False
            if description_from_df is None and description_from_list is None:
                match = True
            elif description_from_df is not None and description_from_list is not None:
                if (
                    str(description_from_df).strip()
                    == str(description_from_list).strip()
                ):
                    match = True

            if not match:
                all_content_match = False
                print(f"Несовпадение на индексе {idx}:")
                print(f"Описание из items_df: '{str(description_from_df)[:100]}...'")
                print(
                    f"Описание из generated_keywords_list: '{str(description_from_list)[:100]}...'"
                )
                break
            else:
                print(f"Совпадение на индексе {idx} подтверждено.")

    if all_content_match and len(generated_keywords_list) == len(items_df):
        print("Проверка содержимого пройдена. Порядок элементов совпадает.")
        keywords_values = [d["generated_keywords"] for d in generated_keywords_list]
        items_df_updated = items_df.with_columns(
            pl.Series(name="generated_keywords", values=keywords_values)
        )
        print("Столбец 'generated_keywords' добавлен в DataFrame.")
        print("Первые 5 строк обновленного DataFrame:")
        columns_to_show = [
            col
            for col in [description_column_name_in_df, "generated_keywords"]
            if col in items_df_updated.columns
        ]
        if "id" in items_df_updated.columns:
            columns_to_show.insert(0, "id")
        print(items_df_updated.select(columns_to_show).head())

    elif len(generated_keywords_list) == len(items_df):
        print("Проверка содержимого не пройдена.")
        print(
            "Порядок элементов, вероятно, не совпадает, несмотря на одинаковую длину."
        )

Длины совпадают: 15963. Выполняется выборочная проверка содержимого...
Проверяем индексы: [0, 7981, 11089, 12137, 14619, 15962]
Совпадение на индексе 0 подтверждено.
Совпадение на индексе 7981 подтверждено.
Совпадение на индексе 11089 подтверждено.
Совпадение на индексе 12137 подтверждено.
Совпадение на индексе 14619 подтверждено.
Совпадение на индексе 15962 подтверждено.
Проверка содержимого пройдена. Порядок элементов совпадает.
Столбец 'generated_keywords' добавлен в DataFrame.
Первые 5 строк обновленного DataFrame:
shape: (5, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ description                     ┆ generated_keywords              │
│ ---                             ┆ ---                             │
│ str                             ┆ str                             │
╞═════════════════════════════════╪═════════════════════════════════╡
│ Мелодрама легендарного Педро А… ┆ Трагедия, любовь, комедия, бол… │
│ Уморительная современная комед… ┆ мальч

In [ ]:
# items_df_updated.write_parquet("items_with_generated_keywords.parquet")

In [ ]:
# Load resulted df
items_df_with_keywords = pl.read_parquet("items_with_generated_keywords.parquet")

In [ ]:
sample_dataset = (
    items_df_with_keywords.sample(2)
    .select("keywords", "countries", "title", "description", "generated_keywords")
    .rows(named=True)
)

for item in sample_dataset:
    print(f"Title: {item['title']}")
    print(f"Description: {item['description']}")
    print(f"Countries: {item['countries'][:100]}")
    print(f"Keywords: {item['keywords'][:150]}...")
    print(f"Keywords: {item['generated_keywords'][:150]}...")
    print("-" * 80)

Title: высоцкий. спасибо, что живой
Description: Главный фильм 2012 года по итогам российского проката, одна из интереснейших кинозагадок и самая громкая после «Бумера» работа Петра Буслова. Зрители до сих пор горячо обсуждают картину «Высоцкий. Спасибо, что живой…». Ну а тем, кто все еще не знает, кто же сыграл заглавную роль, будет вдвойне интересно посмотреть онлайн фильм, посвященный последнему году жизни легенды. Сценарий написал сын певца, Никита Высоцкий. Он же стал одним из продюсеров проекта наряду с Константином Эрнстом. Помимо актера, чье имя почти год держалось в строжайшем секрете, в фильме снялись Оксана Акиньшина, Иван Ургант, Андрей Панин, Максим Леонидов, Владимир Ильин, Сергей Шакуров, Марина Александрова. Картина вызвала широкий общественный резонанс, получив как восторженные, так и разгромные отзывы. Впрочем, лучше один раз увидеть, чем сто раз услышать.  «Высоцкий. Спасибо, что живой…» – это попытка правдиво рассказать о жизни человека, ставшего кумиром не одного п

## Making General Description of an Item


In [ ]:
def clean(text: str, max_len=None):
    text = html.unescape(text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.rstrip(",")
    text = text.strip()
    text = text.lower()

    return text


In [ ]:
items_df_with_keywords_cleaned = (
    items_df_with_keywords
    # Cleaning
    .with_columns(
        [
            pl.col("title").cast(pl.Utf8).map_elements(clean, return_dtype=pl.Utf8),
            pl.col("genres").cast(pl.Utf8).map_elements(clean, return_dtype=pl.Utf8),
            pl.col("generated_keywords")
            .cast(pl.Utf8)
            .map_elements(lambda x: clean(str(x), max_len=120), return_dtype=pl.Utf8),
            pl.col("description")
            .cast(pl.Utf8)
            .map_elements(lambda x: clean(str(x), max_len=400), return_dtype=pl.Utf8),
            pl.col("directors")
            .cast(pl.Utf8)
            .map_elements(lambda x: clean(str(x), max_len=100), return_dtype=pl.Utf8),
            pl.col("actors")
            .cast(pl.Utf8)
            .map_elements(lambda x: clean(str(x), max_len=100), return_dtype=pl.Utf8),
            pl.col("studios")
            .cast(pl.Utf8)
            .map_elements(lambda x: clean(str(x), max_len=100), return_dtype=pl.Utf8),
        ]
    )
    # Concating
    .with_columns(
        pl.concat_str(
            [
                # Title section
                pl.when(pl.col("title").is_not_null())
                .then(pl.lit("title: ") + pl.col("title"))
                .otherwise(None),
                # Genres section
                pl.when(pl.col("genres").is_not_null())
                .then(pl.lit("genres: ") + pl.col("genres"))
                .otherwise(None),
                # Keywords section
                pl.when(pl.col("generated_keywords") != "unknown")
                .then(pl.lit("kw: ") + pl.col("generated_keywords"))
                .otherwise(None),
                # Plot/Description section
                pl.when(pl.col("description") != "-")
                .then(pl.lit("plot: ") + pl.col("description"))
                .otherwise(None),
                # Directors section
                pl.when(pl.col("directors") != "unknown")
                .then(pl.lit("directors: ") + pl.col("directors"))
                .otherwise(None),
                # Actors section
                pl.when(pl.col("actors") != "unknown")
                .then(pl.lit("actors: ") + pl.col("actors"))
                .otherwise(None),
                # Studios section
                pl.when(pl.col("studios") != "unknown")
                .then(pl.lit("studios: ") + pl.col("studios"))
                .otherwise(None),
                # Release year section
                pl.when(pl.col("release_year_range").is_not_null())
                .then(pl.lit("release_year: ") + pl.col("release_year_range"))
                .otherwise(None),
                # Age rating section
                pl.when(pl.col("age_rating").is_not_null())
                .then(pl.lit("age_rating: ") + pl.col("age_rating"))
                .otherwise(None),
            ],
            separator="\n",
            ignore_nulls=True,
        ).alias("embedding_text")
    )
)


In [ ]:
print(items_df_with_keywords_cleaned["embedding_text"][0])

title: поговори с ней
genres: драмы, зарубежные, детективы, мелодрамы
kw: трагедия, любовь, комедия, больница, журналист, тореро, разговор, кома, горе, исцеление
plot: мелодрама легендарного педро альмодовара «поговори с ней» в 2003 году получила премию «оскар» за лучший сценарий. журналист марко берет интервью у знаменитой женщины-тореро лидии и вскоре влюбляется в нее. однако он не успевает признаться ей в любви – их разлучает страшная трагедия: бык на корриде бросается на несчастную лидию, и та впадает в кому. марко дни и ночи проводит у ее постели в больнице, но никаких улучшений не предвидится. вскоре марко знакомится в больнице с бениньо, человеком, который уже не первый год ухаживает за своей возлюбленной алисией, которая тоже лежит в коме. объединенные общим горе мужчины сближаются, и однажды бениньо дает страдающему марко простой совет: «поговори с ней». с тех пор марко ведет с бесчувственной лидией долгие разговоры, которые одни только и могут изменить ситуацию к лучшему и сп

- Выглядит неплохо


In [ ]:
model = SentenceTransformer(
    "jinaai/jina-embeddings-v3", trust_remote_code=True, device="cuda"
)
tok = model.tokenizer
TASK = "retrieval.passage"

In [ ]:
import tqdm

texts = items_df_with_keywords_cleaned["embedding_text"].to_list()
vecs = []

for i in tqdm.tqdm(range(0, len(texts), 256)):
    vecs.append(
        model.encode(
            texts[i : i + 256],
            task=TASK,
            normalize_embeddings=True,
            convert_to_numpy=True,
        ).astype(np.float32)
    )
emb = np.vstack(vecs)

emb_256 = emb[:, :256].copy()

100%|██████████| 63/63 [02:24<00:00,  2.29s/it]


In [ ]:
items_df_with_keywords_cleaned = items_df_with_keywords_cleaned.with_columns(
    pl.Series(
        name="embedding",
        values=emb_256.astype("float32").tolist(),
        dtype=pl.List(pl.Float32),
    )
)

In [ ]:
assert items_df_with_keywords_cleaned["embedding"].dtype == pl.List(pl.Float32)
items_df_with_keywords_cleaned["embedding"][0].shape[0]

256

In [ ]:
# Check metrics
K = 10

emb = np.stack(items_df_with_keywords_cleaned["embedding"].to_numpy())  # (N, 256)
genres_list = items_df_with_keywords_cleaned["genres"].fill_null("").to_list()

# genres в сет для быстрого пересечения
genres_set = [set(g.split(", ")) if g else set() for g in genres_list]

In [ ]:
sim = cosine_similarity(emb)
np.fill_diagonal(sim, -1)

topk_idx = np.argpartition(-sim, K, axis=1)[:, :K]

# Genre Hit‑Rate@K
hits = 0
for i, neigh in enumerate(topk_idx):
    gi = genres_set[i]
    if not gi:
        continue
    if any(gi & genres_set[j] for j in neigh):
        hits += 1

hit_rate = hits / len(items_df_with_keywords_cleaned)
print(f"Genre Hit‑Rate@{K}: {hit_rate:.3f}")

# Средняя внутрижанровая схожесть
pairs_sim = []
for g in tqdm.tqdm(set.union(*genres_set), desc="intra‑genre"):
    # найдём все фильмы с этим жанром
    idx = [i for i, gs in enumerate(genres_set) if g in gs]
    if len(idx) < 2:
        continue
    sub = sim[np.ix_(idx, idx)]
    upper = sub[np.triu_indices_from(sub, k=1)]
    pairs_sim.append(upper.mean())

intra_mean = np.mean(pairs_sim)
print(f"Mean Intra‑Genre Similarity: {intra_mean:.3f}")

Genre Hit‑Rate@10: 0.987


intra‑genre: 100%|██████████| 95/95 [00:01<00:00, 73.87it/s]

Mean Intra‑Genre Similarity: 0.400


# Clustering


In [71]:
import polars as pl
import faiss
from sklearn.cluster import KMeans
from pathlib import Path
import numpy as np
from collections import Counter
from statistics import mean, stdev
import math, random


In [ ]:
path_to_data_wit_embeddinds = (
    Path.cwd().resolve().parent
    / "data"
    / "modified_data"
    / "items_with_embeddings.parquet"
)

iteractions = pl.read_parquet(
    Path.cwd().resolve().parent / "data" / "modified_data" / "interactions.parquet"
)

df = pl.read_parquet(path_to_data_wit_embeddinds)

### k‑NN‑граф через FAISS


- Идея:
  1. Нормализируем вектора, чтобы их скалярное произведение равнялось косинусовым сходством
  2. Создаем обьект индекс, который позволяет искать ближайших соседей и добавляем в него вектора
  3. Ищем ближайших соседей


In [21]:
K = 50
TARGET_L2 = 320
MAX_DEPTH = 9
MAX_ITEMS = 120
MIN_ITEMS = 6

ids = df["item_id"].to_numpy()
vecs = np.stack(df["embedding"].to_numpy()).astype("float32")
faiss.normalize_L2(vecs)  # Normalizing vectors -> (dot product = cos similarity)
index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)


D, I = index.search(vecs, K + 1)

edges = pl.DataFrame(
    {
        "src": np.repeat(ids, K),  # 50 повторений, так как у нас 50 ближайших соседей
        "dst": ids[I[:, 1:].ravel()],  # Cосед + убираем самого себя
        "cosine": D[:, 1:].ravel().astype("float32"),  # Дистанция
    }
)

assert not (edges["src"] == edges["dst"]).any()  # Проверка на self-match

### суммарный трафик per item


In [31]:
watch = iteractions.group_by("item_id").agg(pl.col("total_dur").sum().alias("traffic"))

traffic_map = watch.to_dict(as_series=False)
traffic = {row["item_id"]: row["traffic"] for row in watch.to_dicts()}
default_traffic = 1.0

### balanced‑split рекурсией


In [70]:
total_trf = sum(traffic.get(i, default_traffic) for i in ids)
max_leaf_trf = total_trf / TARGET_L2


# векторизованная карта item_id → embedding индекс
id2idx = {id_: idx for idx, id_ in enumerate(ids)}

clusters = []  # итоговые листья
stack = [(ids.tolist(), 0)]  # (список вершин, depth)

while stack:
    nodes, depth = stack.pop()  # Берем последний элемент из стека
    leaf_trf = sum(
        traffic.get(n, default_traffic) for n in nodes
    )  # Считаем суммарный трафик для листа

    # условие добавления листа в кластер

    # Если достигли максимальной глубины
    # Или суммарный трафик листа меньше максимального
    # Или количество элементов в листе меньше минимального
    if (
        (depth == MAX_DEPTH)
        or (leaf_trf <= max_leaf_trf and len(nodes) <= MAX_ITEMS)
        or (len(nodes) <= MIN_ITEMS)
    ):
        clusters.append(nodes)
        continue

    # KMeans на эмбеддингах узлов
    idx = [id2idx[n] for n in nodes]
    X = vecs[idx]

    km = KMeans(n_clusters=2, n_init="auto", random_state=42).fit(X)
    left = [nodes[i] for i, lbl in enumerate(km.labels_) if lbl == 0]
    right = [n for n in nodes if n not in left]

    # гарантия непустых подкластеров
    if not left or not right:
        clusters.append(nodes)
    else:
        stack.append((left, depth + 1))
        stack.append((right, depth + 1))

print(f"Total L2 clusters: {len(clusters)}")

NameError: name 'ids' is not defined

In [47]:
sizes = [len(c) for c in clusters]
print(
    "Cluster sizes: min={} 25%={} median={} 75%={} max={}".format(
        min(sizes),
        np.percentile(sizes, 25),
        np.median(sizes),
        np.percentile(sizes, 75),
        max(sizes),
    )
)

Cluster sizes: min=1 25%=36.0 median=61.0 75%=87.0 max=178


In [50]:
if "traffic_map" in globals():
    totals = [sum(traffic_map.get(i, 1.0) for i in cl) for cl in clusters]
    cv = stdev(totals) / mean(totals)
    print(f"Traffic coefficient‑of‑variation: {cv:.2f}")
else:
    print("traffic_map not defined -> пропускаем чек трафика")

Traffic coefficient‑of‑variation: 0.56


In [51]:
id2idx = {id_: idx for idx, id_ in enumerate(ids)}
id2cl = {i: cid for cid, cl in enumerate(clusters) for i in cl}


def sample_intra_inter(n_pairs=5000):
    """
    Считает среднее сходство внутри и между кластерами
    Сходство внутри кластера должно быть выше, чем между кластерами
    """

    intra, inter = [], []
    rng = random.Random(42)
    for _ in range(n_pairs):
        a = rng.choice(ids)
        b = rng.choice(ids)
        sim = float(np.dot(vecs[id2idx[a]], vecs[id2idx[b]]))
        (intra if id2cl[a] == id2cl[b] else inter).append(sim)
    return np.mean(intra), np.mean(inter)


intra, inter = sample_intra_inter()
print(f"Mean intra‑sim = {intra:.3f}   |   Mean inter‑sim = {inter:.3f}")

embedding_text = df["embedding_text"].to_list()

# ручной чек кластеров
if "embedding_text" in globals():
    rng = random.Random(0)
    for _ in range(3):
        cid = rng.randrange(len(clusters))
        sample_titles = rng.sample(clusters[cid], k=min(5, len(clusters[cid])))
        print(f"\nCluster {cid}  | size={len(clusters[cid])}")
        for tid in sample_titles:
            txt = embedding_text[tid][:120].replace("\n", " ")
            print("  •", txt + ("…" if len(embedding_text[tid]) > 120 else ""))
else:
    print("embedding_text not defined -> пропускаем ручной осмотр")

Mean intra‑sim = 0.453   |   Mean inter‑sim = 0.282

Cluster 216  | size=47
  • title: автобан (жестовым языком) genres: боевики, зарубежные, триллеры kw: криминальные разборки, сурдоперевод, немецкая…
  • title: дети в интернете (сурдоперевод + субтитры) genres: для детей kw: дети, интернет, приключения, девочка, собака, су…
  • title: смешарики: обучающая азбука genres: мультсериалы, для самых маленьких, мультфильмы kw: смешарики, обучающая азбук…
  • title: эти разные, разные, разные лица genres: советские, приключения, комедии kw: чехов, миниатюры, игорь ильинский, ше…
  • title: затерянный мир (на английском языке) genres: фантастика, зарубежные, фэнтези, приключения kw: немой фильм, диноза…

Cluster 124  | size=99
  • title: пламя - я твоя тень genres: драмы kw: первобытный мир, огонь, борьба за выживание, примитивная пара, драка, выжив…
  • title: таинственный сад genres: семейное, фэнтези, приключения kw: таинственный сад, магическое место, детство, приключе…
  • title: ковчег 

дерево L2 годится для следующего этапа. 249 кластеров равномерны по размеру и трафику, сходство внутри заметно выше, чем между.


### Вытаскиваем ключевые слова из кластера


In [53]:
# Joining item_id and cluster_L2
cl_rows = [
    {"item_id": item, "cluster_L2": cid}
    for cid, cl in enumerate(clusters)
    for item in cl
]
cl_df = pl.DataFrame(cl_rows)

df = df.join(cl_df, on="item_id", how="left")

item_id,cluster_L2
i64,i64
7357,0
9847,0
14411,0
11191,0
2668,0


In [65]:
print(df.filter(pl.col("cluster_L2") == 228).select("title").to_series().to_list())

['легенды саванны', 'джордж из джунглей', 'бунюэль в лабиринте черепах', 'навозанда, музыкант', 'кинг конг', 'тарзан и его подруга', 'маленький будда', 'королевство обезьян', 'кунг-фу панда', 'тарзан и амазонки', 'тарзан находит сына', 'кинг конг', 'тайное сокровище тарзана', 'кинг конг', 'айнбо. сердце амазонии', 'тарзан и охотники с капканами', 'золото джунглей', 'тарзан повелитель обезьян', 'король обезьян', 'тарзан', 'король обезьян', 'тарзан и восстание в джунглях', 'приключение тарзана в нью-йорке', 'динозавр', 'чёрный котёл', 'легенда о нараяме', 'приключения тарзана в нью-йорке', 'спасение тарзана', 'тарзан: повелитель обезьян']


- Прикольно, оч хорошо кластер сформировался. Какие-то вайбы приключений и джунглей! Очень хорошо


In [58]:
# df.write_parquet(Path.cwd().resolve().parent / "data" / "modified_data" / "items_with_clusters.parquet")

# Cluster Keywords


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk, re, html
import polars as pl
from pathlib import Path

nltk.download("punkt", quiet=True)

True

In [2]:
df = pl.read_parquet(
    Path.cwd().resolve().parent
    / "data"
    / "modified_data"
    / "items_with_clusters.parquet"
)


In [3]:
df = df.with_columns(pl.col("generated_keywords").alias("core_text"))

cluster_docs_df = (
    df.group_by("cluster_L2")
    .agg(pl.col("core_text").implode().list.join(" ").alias("doc"))
    .sort("cluster_L2")
)

In [5]:
def tokenize(txt):
    tokens = nltk.word_tokenize(txt, language="russian")
    return [t.lower() for t in tokens if re.match(r"\w{3,}", t)]


vectorizer = TfidfVectorizer(
    tokenizer=tokenize, ngram_range=(1, 2), min_df=2, max_features=50000
)

tfidf = vectorizer.fit_transform(cluster_docs_df["doc"].to_list())

/Users/rustemhutiev/Projects/Phenomena/llm4recsys/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
vocab = np.array(vectorizer.get_feature_names_out())
TOP_N = 5

cluster_docs_df = cluster_docs_df.with_columns(
    pl.Series(
        "keywords",
        [
            ", ".join(vocab[row.toarray().ravel().argsort()[-TOP_N:][::-1]])
            for row in tfidf
        ],
    )
)


In [11]:
cluster_docs_df.select(pl.exclude("doc")).write_parquet(
    Path.cwd().resolve().parent / "data" / "modified_data" / "cluster_labels_df.parquet"
)

# Transitions


In [1]:
interactions_df = pl.read_parquet(
    Path.cwd().resolve().parent / "data" / "modified_data" / "interactions.parquet"
)
cluster_labels_df = pl.read_parquet(
    Path.cwd().resolve().parent / "data" / "modified_data" / "cluster_labels_df.parquet"
)

NameError: name 'pl' is not defined

In [2]:
assert {"user_id", "item_id", "last_watch_dt"}.issubset(interactions_df.columns), (
    "interactions_df lacks mandatory columns"
)

has_pct = "watched_pct" in interactions_df.columns
has_dur = "total_dur" in interactions_df.columns

NameError: name 'interactions_df' is not defined

In [19]:
# helper: id -> cluster -> keywords
item2cl = dict(zip(df["item_id"], df["cluster_L2"]))
cl2kw = dict(zip(cluster_labels_df["cluster_L2"], cluster_labels_df["keywords"]))


def kw(cid):
    return cl2kw.get(cid, f"cluster_{cid}")


# фильтруем хорошие просмотры
QUAL_EXPR = (
    (pl.col("watched_pct") >= 70)
    if "watched_pct" in interactions_df.columns
    else pl.all()
)
vi = (
    interactions_df.filter(QUAL_EXPR)
    .sort(["user_id", "last_watch_dt"])
    .with_columns(
        pl.col("item_id")
        .map_elements(lambda x: item2cl.get(x), return_dtype=pl.Int64)
        .alias("cluster_L2")
    )
    .drop_nulls("cluster_L2")
)

records = []
for uid, grp in vi.group_by("user_id"):
    seq = grp.select(["cluster_L2", "last_watch_dt"]).to_numpy()
    unique = []
    last = None
    for cid, ts in seq:
        if cid != last:
            unique.append((cid, ts))
            last = cid
    for i in range(2, len(unique)):
        c2, _ = unique[i - 2]
        c1, _ = unique[i - 1]
        nxt, _ = unique[i]
        if nxt in (c1, c2):
            continue
        prompt_text = f"""The films or series I watched most recently are in the following clusters:

Cluster 1: {kw(c2)}

Cluster 2: {kw(c1)}

Each cluster is described with salient phrases or entity names.
With less than 30 words, generate a new and different cluster I will watch next
with highly specific salient phrases or entity names, with a prefix that says "New cluster: "
"""
        target_text = f"New cluster: {kw(nxt)}"
        records.append((prompt_text, target_text))

transitions_df = pl.DataFrame(records, schema=["prompt", "target"], orient="row")
print("Transitions shape:", transitions_df.shape)
print(transitions_df.head(2))

Transitions shape: (1188204, 2)
shape: (2, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ prompt                          ┆ target                          │
│ ---                             ┆ ---                             │
│ str                             ┆ str                             │
╞═════════════════════════════════╪═════════════════════════════════╡
│ The films or series I watched … ┆ New cluster: оазис, приключени… │
│ The films or series I watched … ┆ New cluster: принцесса, короле… │
└─────────────────────────────────┴─────────────────────────────────┘


In [21]:
print(transitions_df["prompt"][0])

The films or series I watched most recently are in the following clusters:

Cluster 1: приключения, животные, юмор, комедия, мультфильм

Cluster 2: приключения, мультфильм, попугай, дружба, друзья

Each cluster is described with salient phrases or entity names.
With less than 30 words, generate a new and different cluster I will watch next
with highly specific salient phrases or entity names, with a prefix that says "New cluster: "



In [24]:
# Балансировка классов

target_cnt = transitions_df.group_by("target").len().sort("len", descending=True)

print("Всего уникальных target‑кластеров:", target_cnt.height)
print(target_cnt)


# добавим колонку pair_id, чтобы считать частоту пары
transitions_df = transitions_df.with_columns(
    (pl.col("prompt").hash()).alias("pair_id")  # хэш prompt = пара кластеров
)

# посчитаем частоту каждой pair_id в рамках target
freq = transitions_df.group_by(["target", "pair_id"]).len().rename({"len": "pair_freq"})

df_ = transitions_df.join(freq, on=["target", "pair_id"], how="left")

balanced = (
    df_.sort(["target", "pair_freq"], descending=True)
    .group_by("target")
    .head(10)  # <= 10 примеров на label
    .drop(["pair_id", "pair_freq"])
)

print("До балансировки:", transitions_df.height)
print("После:", balanced.height)

Всего уникальных target‑кластеров: 246
shape: (246, 2)
┌─────────────────────────────────┬───────┐
│ target                          ┆ len   │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ New cluster: месть, драма, бое… ┆ 38948 │
│ New cluster: драма, любовь, бо… ┆ 32761 │
│ New cluster: война, отечествен… ┆ 29560 │
│ New cluster: семейные, семейны… ┆ 28139 │
│ New cluster: расследование, ра… ┆ 26510 │
│ …                               ┆ …     │
│ New cluster: шведский, кардина… ┆ 25    │
│ New cluster: фитнес, тренировк… ┆ 23    │
│ New cluster: игры для, игры, п… ┆ 22    │
│ New cluster: лига чемпионов, ч… ┆ 6     │
│ New cluster: motify, тренировк… ┆ 3     │
└─────────────────────────────────┴───────┘
До балансировки: 1188204
После: 2449


In [26]:
balanced.write_parquet(
    Path.cwd().resolve().parent
    / "data"
    / "modified_data"
    / "balanced_transitions.parquet"
)

In [3]:
import pandas as pd
from pathlib import Path

data = pd.read_parquet(
    Path.cwd().resolve().parent
    / "data"
    / "modified_data"
    / "balanced_transitions.parquet"
)
data = data.rename(columns={"target": "completion"})

In [5]:
print(data["prompt"][0] + data["completion"][0])

The films or series I watched most recently are in the following clusters:

Cluster 1: каледония, новая каледония, океан, биоразнообразие, гондурас

Cluster 2: индонезия, гондурас, природа, папуа, леса

Each cluster is described with salient phrases or entity names.
With less than 30 words, generate a new and different cluster I will watch next
with highly specific salient phrases or entity names, with a prefix that says "New cluster: "
New cluster: южная африка, африка, южная, дикая, дикая природа
